In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import math
import random
from array import array
import time
import matplotlib
import sys

plt.rcParams.update({
'font.size': 25,
'figure.figsize': (11, 10),
'figure.autolayout': True,
'axes.grid': True,
'grid.linestyle': '-',
'grid.alpha': 0.2,
'lines.markersize': 5.0,
'xtick.minor.visible': True,
'xtick.direction': 'in',
'xtick.major.size': 20.0,
'xtick.minor.size': 10.0,
'xtick.top': True,
'ytick.minor.visible': True,
'ytick.direction': 'in',
'ytick.major.size': 20.0,
'ytick.minor.size': 10.0,
'ytick.right': True,
'errorbar.capsize': 0.0
})

x_font_size = 25
y_font_size = 35
l_font_size = 25

In [2]:
Rotors = '''
INPUT 	A 	B 	C 	D 	E 	F 	G 	H 	I 	J 	K 	L	M 	N 	O 	P 	Q 	R 	S 	T 	U 	V 	W	X 	Y 	Z
Rotor I 	E 	K 	M 	F 	L 	G 	D 	Q 	V 	Z 	N 	T	O 	W 	Y 	H 	X 	U 	S 	P 	A 	I 	B	R 	C 	J
Rotor II 	A 	J 	D 	K 	S 	I 	R 	U 	X 	B 	L 	H	W 	T 	M 	C 	Q 	G 	Z 	N 	P 	Y 	F	V 	O 	E
Rotor III 	B 	D 	F 	H 	J 	L 	C 	P 	R 	T 	X 	V	Z 	N 	Y 	E 	I 	W 	G 	A 	K 	M 	U	S 	Q 	O
Rotor IV 	E 	S 	O 	V 	P 	Z 	J 	A 	Y 	Q 	U 	I	R 	H 	X 	L 	N 	F 	T 	G 	K 	D 	C	M 	W 	B
Rotor V 	V 	Z 	B 	R 	G 	I 	T 	Y 	U 	P 	S 	D	N 	H 	L 	X 	A 	W 	M 	J 	Q 	O 	F	E 	C 	K
Rotor VI 	J 	P 	G 	V 	O 	U 	M 	F 	Y 	Q 	B 	E	N 	H 	Z 	R 	D 	K 	A 	S 	X 	L 	I	C 	T 	W
Rotor VII 	N 	Z 	J 	H 	G 	R 	C 	X 	M 	Y 	S 	W	B 	O 	U 	F 	A 	I 	V 	L 	P 	E 	K	Q 	D 	T
Rotor VIII 	F 	K 	Q 	H 	T 	L 	X 	O 	C 	B 	J 	S	P 	D 	Z 	R 	A 	M 	E 	W 	N 	I 	U	Y 	G 	V
Beta rotor 	L 	E 	Y 	J 	V 	C 	N 	I 	X 	W 	P 	B	Q 	M 	D 	R 	T 	A 	K 	Z 	G 	F 	U	H 	O 	S
Gamma rotor 	F 	S 	O 	K 	A 	N 	U 	E 	R 	H 	M 	B	T 	I 	Y 	C 	W 	L 	Q 	P 	Z 	X 	V	G 	J 	D
'''

Reflectors = '''
reflector B	(AY) (BR) (CU) (DH) (EQ) (FS) (GL) (IP) (JX) (KN) (MO) (TZ) (VW)
reflector C	(AF) (BV) (CP) (DJ) (EI) (GO) (HY) (KR) (LZ) (MX) (NW) (TQ) (SU)
reflector B Dünn	(AE) (BN) (CK) (DQ) (FU) (GY) (HW) (IJ) (LO) (MP) (RX) (SZ) (TV)
reflector C Dünn	(AR) (BD) (CO) (EJ) (FN) (GT) (HK) (IV) (LM) (PW) (QZ) (SX) (UY)
'''


In [3]:
Rotors_list = []
for i in range(0, len(Rotors.split('\n'))):
    if (Rotors.split('\n')[i] != ''):
        Rotors_list.append(Rotors.split('\n')[i].replace(' ','').split('\t'))
        
Rotors_dict_indexed = {}
for i in range (0,len(Rotors_list)):
    res = {Rotors_list[0][1:][j]: Rotors_list[i][1:][j] for j in range(len(Rotors_list[0][1:]))}
    Rotors_dict_indexed[i] = res

In [4]:
def rotor(symbol, n, reverse=False):
    line = ''
    if (reverse):
        for letter in symbol.replace(' ',''):
            line += {v:k  for k,v in Rotors_dict_indexed[n].items()}[letter]
    else:
        for letter in symbol.replace(' ',''):
            line += Rotors_dict_indexed[n][letter]
    return line

In [5]:
Reflectors_list = []
Reflectors_list.append(Rotors_dict_indexed[0])
for i in range(0, len(Reflectors.split('\n'))):
    if (Reflectors.split('\n')[i] != ''):
        res1 = {Reflectors.split('\n')[i].split('\t')[1].split(' ')[j][1]:Reflectors.split('\n')[i].split('\t')[1].split(' ')[j][2] for j in range(0,len(Reflectors.split('\n')[i].split('\t')[1].split(' ')))}
        res2 = {Reflectors.split('\n')[i].split('\t')[1].split(' ')[j][2]:Reflectors.split('\n')[i].split('\t')[1].split(' ')[j][1] for j in range(0,len(Reflectors.split('\n')[i].split('\t')[1].split(' ')))}
        res = {**res1, **res2}
        Reflectors_list.append(res)

In [6]:
def reflector(symbol, n):
    line = ''
    for letter in symbol.replace(' ',''):
        line += Reflectors_list[n][letter]
    return line

In [7]:
def caesar(text, shift):
    letter_to_number = {Rotors_list[0][1:][j] : j for j in range(0, len(Rotors_list[0][1:]))}
    number_to_letter = {j : Rotors_list[0][1:][j] for j in range(0, len(Rotors_list[0][1:]))}
    line = ''
    for letter in text.replace(' ',''):
        number = letter_to_number[letter]
        line  += number_to_letter[(number+shift)%26]
    return line

In [174]:
# def enigma(text, ref, rot1, shift1, rot2, shift2, rot3, shift3):
#     text = text.upper()
#     text = text.replace(' ','')
#     text_crypted = ''
#     for i in range(0,len(text)):
#         shift1 = shift1%26
#         shift2 = shift2%26
#         shift3 = shift3%26
#         print('0',shift1, shift2, shift3)
#         if (((rot2 == 1) & (shift3 == (17-1)) & (shift2 != 0)) | ((rot2 == 2) & (shift2 == (5-1)) & (shift2 != 0)) | ((rot2 == 3) & (shift2 == (22-1)) & (shift2 != 0))):
#             shift2 += 1
#             shift1 += 1
#             print('2',shift1, shift2, shift3)
#         shift3 += 1
#         if (((rot3 == 1) & (shift3 == 17) & (shift3 != 0)) | ((rot3 == 2) & (shift3 == 5) & (shift3 != 0)) | ((rot3 == 3) & (shift3 == 22) & (shift3 != 0))):
#             shift2 += 1
#             print('3',shift1, shift2, shift3)
#         forward_shifted = caesar(text[i],shift3)
#         forward_out_3 = rotor(forward_shifted,rot3,False)
#         forward_out_3_shifted = caesar(forward_out_3,shift2-shift3)
#         forward_out_2 = rotor(forward_out_3_shifted,rot2,False)
#         forward_out_2_shifted = caesar(forward_out_2,shift1-shift2)
#         forward_out_1 = rotor(forward_out_2_shifted,rot1,False)
#         forward_out_1_shifted = caesar(forward_out_1,-shift1)
#         reflection_out = reflector(forward_out_1_shifted,ref)
#         reflection_out_shifted = caesar(reflection_out,shift1)
#         backward_out_1 = rotor(reflection_out_shifted,rot1,True)
#         backward_out_1_shifted = caesar(backward_out_1,shift2-shift1)
#         backward_out_2 = rotor(backward_out_1_shifted,rot2,True)
#         backward_out_2_shifted = caesar(backward_out_2,shift3-shift2)
#         backward_out_3 = rotor(backward_out_2_shifted,rot3,True)
#         backward_out_3_shifted = caesar(backward_out_3,-shift3)
#         text_crypted += backward_out_3_shifted
#         print(backward_out_3_shifted)
#     return text_crypted

In [180]:
def enigma(text, ref, rot1, shift1, rot2, shift2, rot3, shift3):
    text = text.upper()
    text = text.replace(' ','')
    text_crypted = ''
    for i in range(0,len(text)):
        if (((rot2 == 1) & (shift3 == (17-1)) & (shift2 != 0)) | ((rot2 == 2) & (shift2 == (5-1)) & (shift2 != 0)) | ((rot2 == 3) & (shift2 == (22-1)) & (shift2 != 0))):
            shift2 += 1
            shift1 += 1
        shift3 += 1
        if (((rot3 == 1) & (shift3 == 17) & (shift3 != 0)) | ((rot3 == 2) & (shift3 == 5) & (shift3 != 0)) | ((rot3 == 3) & (shift3 == 22) & (shift3 != 0))):
            shift2 += 1
        shift1 = shift1%26
        shift2 = shift2%26
        shift3 = shift3%26
        forward_shifted = caesar(text[i],shift3)
        forward_out_3 = rotor(forward_shifted,rot3,False)
        forward_out_3_shifted = caesar(forward_out_3,shift2-shift3)
        forward_out_2 = rotor(forward_out_3_shifted,rot2,False)
        forward_out_2_shifted = caesar(forward_out_2,shift1-shift2)
        forward_out_1 = rotor(forward_out_2_shifted,rot1,False)
        forward_out_1_shifted = caesar(forward_out_1,-shift1)
        reflection_out = reflector(forward_out_1_shifted,ref)
        reflection_out_shifted = caesar(reflection_out,shift1)
        backward_out_1 = rotor(reflection_out_shifted,rot1,True)
        backward_out_1_shifted = caesar(backward_out_1,shift2-shift1)
        backward_out_2 = rotor(backward_out_1_shifted,rot2,True)
        backward_out_2_shifted = caesar(backward_out_2,shift3-shift2)
        backward_out_3 = rotor(backward_out_2_shifted,rot3,True)
        backward_out_3_shifted = caesar(backward_out_3,-shift3)
        text_crypted += backward_out_3_shifted
    return text_crypted

In [181]:
enigma('AAAAA AAAAA AAAAA AAAAA AAAAA AAAAA AAAAA AAAAA AAAAA AAAAA AAAAA', 1, 2, 3, 2, 3, 2, 3)

'BGDMBTZUONCIZMORCPNVLGOVLMURTNZNDROPETXLPLYCMIBICXITUCM'

In [182]:
enigma('xyz', 1, 3, 1, 1, 2, 2, 3)

'OLC'

In [183]:
enigma('AAAAAAA', 1, 1, 0, 2, 0, 3, 0)

'BDZGOWC'

In [184]:
enigma('BDZGOWC', 1, 1, 0, 2, 0, 3, 0)

'AAAAAAA'

In [185]:
enigma('AAAAA AAAAA', 1, 1, 0, 2, 0, 2, 0)

'SMGJHIUWUF'

In [186]:
enigma('AAAAA AAAAA', 1, 1, 0, 2, 0, 1, 0)

'ZEMUOFUVNB'

In [188]:
enigma('AAAAA AAAAA', 1, 1, 0, 2, 0, 1, 0)

'ZEMUOFUVNB'

In [194]:
enigma('some random text', 1, 2, 0, 2, 0, 2, 0)

'ZBRBISXLIWGXAA'

In [196]:
enigma('some random text what is wrong', 1, 1, 0, 2, 0, 3, 0)

'JIKWEWRUMVMTMAFGDIDTHQHQL'